Dataset
- https://www.kaggle.com/datasets/dzzlr07/absa-natural-tourist-attractions-review/data

In [ ]:
!pip install deep-translator
!pip install datasets

In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd
import numpy as np
import re

Load dataset

In [ ]:
df = pd.read_csv('tourist_reviews.csv')

Function Determine Sentiment

In [ ]:
def determine_sentiment(rating, accessibility, activity, facility):
    if rating >= 4:
        if accessibility == 'negative':
            return 'negative'
        if activity == 'negative':
            return 'negative'
        if facility == 'negative':
            return 'negative'
        if facility == 'neutral' and activity == 'neutral' and accessibility == 'neutral':
            return 'negative'
        return 'positive'
    elif rating <= 3:
        return 'negative'

Cleaning dataset

In [ ]:
df = df.drop('id', axis=1)
df = df.drop('datetime', axis=1)
df = df.drop('location', axis=1)

df['sentiment'] = df[['rating', 'accessibility', 'activity', 'facility']].apply(lambda row: determine_sentiment(*row), axis=1)

df = df.drop('rating', axis=1)
df = df.drop('accessibility', axis=1)
df = df.drop('facility', axis=1)
df = df.drop('activity', axis=1)

df_sampled = df.groupby('sentiment').apply(lambda x: x.sample(n=100)).reset_index(drop=True) # only add 1000 positive and 1000 negative
df_sampled = df_sampled.sample(frac=1).reset_index(drop=True) # randomized
df_clean_dataset = df_sampled.rename(columns={'text': 'reviews_content', 'sentiment': 'category'})
df_sampled.to_csv('tourist_reviews_clean.csv', index=False)


Translated Dataset to English

In [ ]:
df1 = pd.read_csv('tourist_reviews_clean.csv')

In [ ]:
from deep_translator import GoogleTranslator

def translate_to_english(text):
    return GoogleTranslator(source='auto', target='en').translate(text)

df1['reviews_content'] = df1['reviews_content'].apply(translate_to_english)

df1.to_csv('tourist_reviews_final.csv', index=False)

DataTest Model

In [ ]:
# Function to clean the specific noise from the text
def clean_text(text):
    # Remove remains of emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove remains of user mentions
    text = re.sub(r'@\S+', '', text)
    # Remove Twitter links
    text = re.sub(r'http\S+', '', text)
    # Remove retweet identifiers
    text = re.sub(r'\bRT\b', '', text)
    text = re.sub(r'\[RE.*?\]', '', text)
    # Remove HTML entities like &amp;
    text = re.sub(r'&\S+;', '', text)
    # Remove hashtags
    text = re.sub(r'#\S+', '', text)
    return text

Load DataTest

In [ ]:
# read csv
df = pd.read_csv('hafiiid.csv')

In [ ]:
df.head()

In [ ]:
# drop row 
df = df.drop('Place Name', axis=1)
df = df.drop('quote_count', axis=1)
df = df.drop('favorite_count', axis=1)
df = df.drop('reply_count', axis=1)
df = df.drop('retweet_count', axis=1)

# Convert 'created_at' to datetime
df['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')

# Extract month and create a new column 'month'
df['month'] = df['created_at'].dt.month

df = df.drop('created_at', axis=1)
df = df.sort_values(['id', 'month'])
df['reviews_content'] = df['full_text'].apply(clean_text)
df = df.drop('full_text', axis=1)
df_new1  = df[['reviews_content']]
df_new2  = df[['id','month']]

# convert to csv
df_new1.to_csv('datatest_clean.csv', index=False)
df_new2.to_csv('id_month.csv', index=False)

Translated DataTest to English

In [ ]:
df1 = pd.read_csv('datatest_clean.csv')

In [ ]:
from deep_translator import GoogleTranslator

def translate_to_english(text):
    return GoogleTranslator(source='auto', target='en').translate(text)

df1['reviews_content'] = df1['reviews_content'].apply(translate_to_english)

df1.to_csv('datatest_final.csv', index=False)